<a href="https://colab.research.google.com/github/yussouf88/Deep_learning/blob/main/my_new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import numpy as numpy
import io
import torch
from google.colab import files
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [66]:
ur1 = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
df = pd.read_csv(ur1)

In [112]:
class Model(nn.Module):
  def __init__(self,in_features=4,h1=8,h2=9,out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)

  def forward(self,x):
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.out(x)
    return x

torch.manual_seed(32)
model = Model()

In [95]:
class my_class(nn.Module):
  def __init__ (self,in_features=4,h1=8,h2=9,h3=8,out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.fc3 = nn.Linear(h2,h3)
    self.out = nn.Linear(h3,out_features)

  def forward(self,x):
    x = f.relu(self,fc1(x))
    x = f.relu(self,fc2(x))
    x = f.relu(self,fc3(x))
    x = self.out(x)
    return x

torch.manual_seed(41)
model = my_class()

In [84]:
df['variety'] = df.species.replace(['setosa','versicolor','virginica'],[0,1,2])
x = df[['sepal_length',	'sepal_width',	'petal_length',	'petal_width']]
y = df['variety']

<ipython-input-84-2282622107>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['variety'] = df.species.replace(['setosa','versicolor','virginica'],[0,1,2])


In [86]:
x = x.values
y = y.values


In [91]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,random_state=10)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [117]:
critereon = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=.1)
epochs = 200
losses = []
for i in range(epochs):
  y_pred = model.forward(x_train)
  loss = critereon(y_pred,y_train)
  losses.append(loss.detach().numpy())
  if i%10==0:
    print(f'epoch {i} loss={loss}')

optimizer.zero_grad()
loss.backward
optimizer.step()


epoch 0 loss=1.174389123916626
epoch 10 loss=1.174389123916626
epoch 20 loss=1.174389123916626
epoch 30 loss=1.174389123916626
epoch 40 loss=1.174389123916626
epoch 50 loss=1.174389123916626
epoch 60 loss=1.174389123916626
epoch 70 loss=1.174389123916626
epoch 80 loss=1.174389123916626
epoch 90 loss=1.174389123916626
epoch 100 loss=1.174389123916626
epoch 110 loss=1.174389123916626
epoch 120 loss=1.174389123916626
epoch 130 loss=1.174389123916626
epoch 140 loss=1.174389123916626
epoch 150 loss=1.174389123916626
epoch 160 loss=1.174389123916626
epoch 170 loss=1.174389123916626
epoch 180 loss=1.174389123916626
epoch 190 loss=1.174389123916626
